In [ ]:
import matplotlib.pyplot as plt

from ALLCools.clustering import tsne
from ALLCools.plot import *
from ALLCools.integration import SeuratIntegration
import wmb

import scanpy as sc
import anndata
from harmonypy import run_harmony

## Parameters

In [ ]:
ref_region_type = 'CEMBA'

## Load

In [ ]:
adata_merge = anndata.read_h5ad('./adata/final.h5ad')
adata_merge

## Harmony for embedding

In [ ]:
ho = run_harmony(data_mat=adata_merge.obsm['X_pca_integrate'],
                 meta_data=adata_merge.obs,
                 nclust=50,
                 vars_use=['Modality'], 
                 max_iter_harmony=30)

In [ ]:
adata_merge.obsm['X_harmony'] = ho.Z_corr.T

In [ ]:
adata_merge

### TSNE

In [ ]:
tsne(adata_merge, obsm='X_harmony')

### UMAP

In [ ]:
adata_merge.obsm['X_pca'] = adata_merge.obsm['X_harmony']

sc.pp.neighbors(adata_merge)

In [ ]:
min_dist = max(0.1, 1 - adata_merge.shape[0] / 60000)
sc.tl.umap(adata_merge, min_dist=min_dist)
del adata_merge.obsm['X_pca']

### Clustering

In [ ]:
sc.tl.leiden(adata_merge, resolution=0.3)

## Plot

In [ ]:
adata_merge.obs

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(10, 5),dpi=300)

merfish_adata = adata_merge[adata_merge.obs['Modality'] == 'merfish']
mC_adata = adata_merge[adata_merge.obs['Modality'] == 'mC']

ax = axes[0]
categorical_scatter(data=merfish_adata,
                        coord_base='tsne',
                        max_points=None,
                        hue=None,
                        scatter_kws=dict(color='lightgrey'),
                        ax=ax)

_ = categorical_scatter(
    data=mC_adata,
    ax=ax,
    coord_base='tsne',
    hue='L1_annot',
    text_anno='L1_annot',
    palette='tab20',
    max_points=None,)

ax.set(title = 'mC L1 annot')

    
    
ax = axes[1]
categorical_scatter(data=mC_adata,
                        coord_base='tsne',
                        max_points=None,
                        hue=None,
                        scatter_kws=dict(color='lightgrey'),
                        ax=ax)

_ = categorical_scatter(
    data=merfish_adata,
    ax=ax,
    coord_base='tsne',
    hue='L1_annot_transfer',
    text_anno='L1_annot_transfer',
    palette='tab20',
    max_points=None,
)
ax.set(title = 'merfish L1 transferred annot')


In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(10, 5),dpi=300)

merfish_adata = adata_merge[adata_merge.obs['Modality'] == 'merfish']
mC_adata = adata_merge[adata_merge.obs['Modality'] == 'mC']

ax = axes[0]
categorical_scatter(data=merfish_adata,
                        coord_base='tsne',
                        max_points=None,
                        hue=None,
                        scatter_kws=dict(color='lightgrey'),
                        ax=ax)

_ = categorical_scatter(
    data=mC_adata,
    ax=ax,
    coord_base='tsne',
    hue='DissectionRegion',
    #text_anno='DissectionRegion',
    palette=wmb.brain.get_dissection_region_palette(region_type=ref_region_type),
    max_points=None,)

ax.set(title = 'mC L1 DissectionRegion')

    
    
ax = axes[1]
categorical_scatter(data=mC_adata,
                        coord_base='tsne',
                        max_points=None,
                        hue=None,
                        scatter_kws=dict(color='lightgrey'),
                        ax=ax)

_ = categorical_scatter(
    data=merfish_adata,
    ax=ax,
    coord_base='tsne',
    hue='DissectionRegion_transfer',
    #text_anno='DissectionRegion_transfer',
    palette=wmb.brain.get_dissection_region_palette(region_type=ref_region_type),
    max_points=None,
)
ax.set(title = 'merfish L1 transferred DissectionRegion')


## Save

In [ ]:
adata_merge.write_h5ad('./adata/final_with_coords.h5ad')

In [ ]:
adata_merge